## 1. Load libraries

In [63]:
import networkit as nk
# https://networkit.github.io/features.html

In [3]:
import csv
import json
from operator import itemgetter
import networkx as nx
from networkx.algorithms import community

In [4]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

import glob, os
os.chdir("/Users/choyademacbook/Desktop/2020Spring/IMT575/finalProject")

## 2. Import data

**- filter user**

user from reviews in 2018

In [67]:
user_2018 = pd.read_csv("./dataset/Yelp_dataset/2_reviews_2018.csv", encoding='utf-8', usecols=["user_id"])
user_2018.shape

(676136, 1)

In [71]:
user_2018 = pd.DataFrame({'user_id': user_2018.user_id.unique()})
user_2018.shape

(321599, 1)

user from reviews in 2019

In [72]:
user_2019 = pd.read_csv("./dataset/Yelp_dataset/2_reviews_2019.csv", encoding='utf-8', usecols=["user_id"])
user_2019.shape

(557262, 1)

In [73]:
user_2019 = pd.DataFrame({'user_id': user_2019.user_id.unique()})
user_2019.shape

(289811, 1)

user were active in 2018 or 2019

In [75]:
user = user_2018.append(user_2019)
user = pd.DataFrame({'user_id': user.user_id.unique()})
user.shape

(538592, 1)

**- join with dataset user**

In [79]:
df_user = pd.read_json('./dataset/Yelp_dataset/yelp_academic_dataset_user.json', lines=True)

In [80]:
df_user = df_user.loc[df_user.user_id.isin(user.user_id)]

In [81]:
df_user.shape

(538592, 22)

## 3. Extract features
- change data types

In [82]:
# change the data type
df_user["yelping_since"]=pd.to_datetime(df_user["yelping_since"])
df_user["since_year"]=df_user["yelping_since"].dt.year
df_user["since_month"]=df_user["yelping_since"].dt.month
df_user["period"]= 2020 - df_user["since_year"]   # being users for X years

- aggregate

In [83]:
# new variables (change the data type to list and then calculate the length of list)
df_user["elite_count"]= df_user['elite'].str.strip('()').str.split(',').str.len()
df_user["friends_count"]= df_user['friends'].str.strip('()').str.split(',').str.len()
df_user['friends']  = df_user.friends.str.strip('()').str.split(',')

# sum up
complimentList = ['compliment_cool', 'compliment_cute','compliment_funny', 
                  'compliment_hot', 'compliment_list','compliment_more', 
                  'compliment_note', 'compliment_photos','compliment_plain',
                  'compliment_profile', 'compliment_writer']
votesList = ['cool', 'funny', 'useful']

df_user["sum_compliment"] = df_user.loc[:,complimentList].sum(axis=1)
df_user["sum_votes"] = df_user.loc[:,votesList].sum(axis=1)

In [84]:
df_user.head(3)

,average_stars,compliment_cool,compliment_cute,compliment_funny,compliment_hot,compliment_list,compliment_more,compliment_note,compliment_photos,compliment_plain,...,useful,user_id,yelping_since,since_year,since_month,period,elite_count,friends_count,sum_compliment,sum_votes
1,3.83999999999999985789,63,2,63,36,1,4,33,5,37,...,790,FOBRPlBHa3WPHFB5qYDlVg,2008-04-28 01:29:25,2008,4,12,6,213,270,1506
7,3.62999999999999989342,623,35,623,483,24,81,193,172,541,...,4127,f4_MRNHvN-yRn7EA8YWRxg,2011-01-17 00:18:23,2011,1,9,8,755,3130,9451
12,3.37000000000000010658,135,13,135,212,7,11,120,72,150,...,2993,q-v8elVPvKz0KvK69QSj1Q,2009-05-19 01:42:25,2009,5,11,8,1855,908,6106


- networks

In [85]:
#temp_df = df_user.iloc[:100, ]

userNetwork_edges = pd.DataFrame({'user':df_user.user_id.repeat(df_user.friends.str.len()),
                                  'friend':np.concatenate(df_user.friends.values)})
userNetwork_edges.shape

(33073340, 2)

In [86]:
# friends are the users who have wrote reviews in 2018

#userNetwork_edges.head()
userNetwork_edges = userNetwork_edges.loc[userNetwork_edges.friend.isin(user_2018.user_id)]
userNetwork_edges.shape

(25937, 2)

In [87]:
userNetwork_edges.to_csv("./temp_dataset/userNetwork_edges.csv", index = False)

In [88]:
# df to a list of tuples
edges = userNetwork_edges.to_records(index=False)
edges = list(edges)
edges

[('QWxW_pApmpY9iPApC8P-UQ', 'hizGc5W1tBHPghM5YKCAtg'),
 ('6ryJ1udGnl7jvgb3HWID3Q', 'ozD_lbZXomRZOABnWX4n5g'),
 ('KVbMZV-XJPSH9wXEwuXIaA', 'zy4NqQpPWNRftutQEhVb1g'),
 ('CXJqjeJ5ZwcXSvX8o9US-g', '3fhIgNqd1jq_4sTWwA79Xg'),
 ('Vnh2IPSRdvwAbpGhiJgw_w', '18bWkZ8pHQqXwa8IvhotIg'),
 ('crsOC96K5ljAgsipGOCKmg', 'sYauKfRBquRxDwHA1a8EsA'),
 ('SCo1UBoeN3bhRMkSYuiX1A', '7QVNLrJDkjsDsCtODlPdsw'),
 ('qoS4nIc5MlfF_j8DkYHBgw', 'Q8z64r0b6fsUpHqkDYevCg'),
 ('fly4T2PUvDd3w1wWfxLhuA', 'XopP_70iyfC3iCPAY-sz4Q'),
 ('pZ6P_1V-bTSoYG1fg1TXzw', '1TMET735B8WOzpDZtj3miA'),
 ('7AMPGltlqe0cxYfIAZATCQ', 'cufhYFdyrUAt4SWe-m0grQ'),
 ('Rx-lnDQMrVgqnI0Pcb2Q5w', '3cBVCNsm273EJ_GM4FVvJg'),
 ('OlUAxKU9RdLOwOVJR-_5jQ', 'hS1i4ZL4RHIAMZHxwDGreA'),
 ('DoRCeCcJbrsM2BiAKj3trA', 'cJh4F1zFNJb2wXCNT1uNWQ'),
 ('cwRZjY-RrP78swBEaevMLg', '3xORNxbKdd08LUf0tclx3A'),
 ('xDed_RrQv7xuwOp9hESt9Q', 'Wu0yySWcHQ5tZ_59HNiamg'),
 ('eVBJgxQg-XRMImQFYwta-Q', '4wp4XI9AxKNqJima-xahlg'),
 ('kIV04ZOFnyLcK93XUQh7eQ', 'tM84r9rP7XTjX4SnRzviMA'),
 ('sFJjEc7

In [89]:
G = nx.Graph()

# add nodes and edges
G.add_nodes_from(list(df_user['user_id']))
G.add_edges_from(edges)

#G = nx.from_pandas_edgelist(userNetwork_edges, source='user', target='friend')

In [90]:
print(nx.info(G))

Name: 
Type: Graph
Number of nodes: 538592
Number of edges: 25365
Average degree:   0.0942


In [91]:
# Eigenvector centrality
# https://cambridge-intelligence.com/keylines-faqs-social-network-analysis/
EGcentrality = nx.eigenvector_centrality_numpy(G)
userCentrality = pd.DataFrame.from_dict(EGcentrality, orient='index')
userCentrality = userCentrality.reset_index()
userCentrality.columns = ['user_id','centrality']

In [92]:
userCentrality.head(5)

,user_id,centrality
0,FOBRPlBHa3WPHFB5qYDlVg,0.00000000000000000000
1,f4_MRNHvN-yRn7EA8YWRxg,-0.00000000000000000001
2,q-v8elVPvKz0KvK69QSj1Q,0.00000000000000000354
3,6bbHSJ0PrgSxh7e5nigKMw,0.00000000000000000001
4,4VmuXuSRhv5UxYUy3tMpiQ,-0.00000000000000000000


In [93]:
userCentrality.shape

(538592, 2)

In [94]:
d = userCentrality.centrality.describe()
pd.options.display.float_format = '{:.20f}'.format
print(d)

count   538592.00000000000000000000
mean         0.00001999905550050701
std          0.00136245972096769191
min         -0.00000000000000134732
25%         -0.00000000000000000001
50%          0.00000000000000000000
75%          0.00000000000000000001
max          0.70694509634849456159
Name: centrality, dtype: float64


- join with other variables

In [95]:
result = pd.merge(userCentrality,
                 df_user[['average_stars', 'user_id', 'review_count', 'fans', 'period', 'sum_compliment', 'sum_votes']],
                 on='user_id')
result.head()

,user_id,centrality,average_stars,review_count,fans,period,sum_compliment,sum_votes
0,FOBRPlBHa3WPHFB5qYDlVg,0.00000000000000000000,3.83999999999999985789,564,27,12,270,1506
1,f4_MRNHvN-yRn7EA8YWRxg,-0.00000000000000000001,3.62999999999999989342,822,137,9,3130,9451
2,q-v8elVPvKz0KvK69QSj1Q,0.00000000000000000354,3.37000000000000010658,666,197,11,908,6106
3,6bbHSJ0PrgSxh7e5nigKMw,0.00000000000000000001,2.20999999999999996447,15,1,9,2,25
4,4VmuXuSRhv5UxYUy3tMpiQ,-0.00000000000000000000,3.87999999999999989342,106,7,12,45,470


In [96]:
result.to_csv("./temp_dataset/user.csv", index=False)